# DIGI405-19S2 - Topic Modeling TED.com transcripts

This notebook introduces Gensim for topic modeling. The TED.com transcripts are available for download on the datasets page on Learn. For the lab we will use the 2018 transcripts.

We are only supporting this notebook using the lab workrooms for DIGI405. 

You should start by installing gensim via the Anaconda Prompt:
`pip install gensim`

Work through the notebook. The key things to do are:
1. to try training some different size models (e.g. 10 topics, 30 topics, 50 topics);  
2. to explore the topic assignments for documents and see what similar topics are returned;  
3. to keep notes on your observations of different models and the results they produce.

Since we need to evaluate topic models against a use case - think about the idea of a recommendation engine: what model performs best for finding similiar TED talks?

In [12]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models.wrappers import LdaMallet
from gensim.models.coherencemodel import CoherenceModel
from gensim import similarities

import os.path
import re
import glob

mallet_path = 'C:\\Mallet\\mallet-2.0.8\\bin\\mallet' # this should be the correct path for the DIGI405 lab workrooms

In [13]:
# to install nltk run this in Anaconda prompt: pip install nltk 
# note if you get an error with stopwords below then uncomment the following lines and rerun this cell 
import nltk
nltk.download('stopwords')

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wzh110\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The following cells contain functions to load a corpus from a directory of text files, preprocess the corpus and create the bag of words document-term matrix. 

In [14]:
def load_data_from_dir(path):
    file_list = glob.glob(path + '/*.txt')

    # create document list:
    documents_list = []
    for filename in file_list:
        with open(filename, 'r', encoding='utf8') as f:
            text = f.read()
            f.close()
            documents_list.append(text)
    print("Total Number of Documents:",len(documents_list))
    return documents_list

In [15]:
def preprocess_data(doc_set,extra_stopwords = {}):
    # adapted from https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python
    # replace all newlines or multiple sequences of spaces with a standard space
    doc_set = [re.sub('\s+', ' ', doc) for doc in doc_set]
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    # add any extra stopwords
    if (len(extra_stopwords) > 0):
        en_stop = en_stop.union(extra_stopwords)
    
    # list for tokenized documents in loop
    texts = []
    # loop through document list
    for i in doc_set:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # add tokens to list
        texts.append(stopped_tokens)
    return texts

In [16]:
def prepare_corpus(doc_clean):
    # adapted from https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python
    # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
    dictionary = corpora.Dictionary(doc_clean)
    
    dictionary.filter_extremes(no_below=5, no_above=0.5)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    # generate LDA model
    return dictionary,doc_term_matrix

## Load and pre-process the corpus
Load the corpus, preprocess with additional stop words and output dictionary and document-term matrix.

In [17]:
# adjust the path below to wherever you have the transcripts2018 folder
document_list = load_data_from_dir("P:\\Downloads\\ted-transcripts\\transcripts")

# I've added extra stopwords here in addition to NLTK's stopword list - you could look at adding others.
doc_clean = preprocess_data(document_list,{'laughter','applause'})
dictionary, doc_term_matrix = prepare_corpus(doc_clean)

Total Number of Documents: 3603


In [18]:
print(dictionary)

Dictionary(23039 unique tokens: ['000', '1', '10', '100', '11']...)


In [73]:
def print_file_from_dir(path):
    file_list = glob.glob(path + '/*.txt')
    
    # create document list:
    documents_list = []
    for filename in file_list:
        documents_list.append(filename)
    print("Total Number of Documents:",len(documents_list))
    return documents_list

document_list2 = print_file_from_dir("P:\\Downloads\\ted-transcripts\\transcripts")

video1 = 'P:\\Downloads\\ted-transcripts\\transcripts\\2010-07-15-tim_jackson_s_economic_reality_check.txt'
video2 = 'P:\\Downloads\\ted-transcripts\\transcripts\\2017-09-20-zeynep_tufekci_we_re_building_a_dystopia_just_to_make_people_click_on_ads.txt'

doc_id = document_list2.index(video1)
print(doc_id)
doc_id2=document_list2.index(video2)
print(doc_id2)
    
# index of document to explore
print(document_list2[doc_id]) 
# index of document to explore
print(document_list2[doc_id2]) 

Total Number of Documents: 3603
2476
2037
P:\Downloads\ted-transcripts\transcripts\2010-07-15-tim_jackson_s_economic_reality_check.txt
P:\Downloads\ted-transcripts\transcripts\2017-09-20-zeynep_tufekci_we_re_building_a_dystopia_just_to_make_people_click_on_ads.txt


In [19]:
text = "So when people voice fears";
for doc in document_list:
    if text in doc:
        print(document_list.index(doc))

2037


In [20]:
text = "I want to talk to you today about prosperity";
for doc in document_list:
    if text in doc:
        print(document_list.index(doc))


2476


## LDA model with 100,60,50,30,20 topics
The following cell sets the number of topics we are training the model for. The one after trains the model and outputs the topics. Note: this can take a while!

In [21]:
number_of_topics=100 # adjust this to alter the number of topics
words=20 #adjust this to alter the number of words output for the topic below

In [22]:
%%time
# runs LDA using Mallet from gensim using the number_of_topics specified above
ldamallet100 = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=number_of_topics, id2word=dictionary)

Wall time: 2min 52s


In [23]:
ldamallet100 = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=number_of_topics, id2word=dictionary)

In [24]:
# output the topics
ldamallet100.show_topics(num_topics=number_of_topics,num_words=words)

[(0,
  '0.049*"dna" + 0.026*"bacteria" + 0.023*"genetic" + 0.023*"cell" + 0.021*"gene" + 0.020*"genes" + 0.017*"genome" + 0.015*"biology" + 0.013*"cells" + 0.011*"proteins" + 0.011*"organisms" + 0.009*"sequence" + 0.009*"human" + 0.009*"protein" + 0.008*"biological" + 0.008*"molecular" + 0.007*"code" + 0.007*"organism" + 0.007*"living" + 0.007*"molecules"'),
 (1,
  '0.060*"india" + 0.026*"village" + 0.018*"indian" + 0.018*"started" + 0.010*"000" + 0.009*"country" + 0.009*"place" + 0.009*"coming" + 0.008*"prize" + 0.008*"factory" + 0.008*"rural" + 0.008*"call" + 0.008*"villages" + 0.007*"10" + 0.007*"poor" + 0.007*"pakistan" + 0.007*"thought" + 0.006*"days" + 0.006*"nobel" + 0.006*"20"'),
 (2,
  '0.087*"energy" + 0.030*"oil" + 0.029*"power" + 0.022*"solar" + 0.020*"electricity" + 0.018*"nuclear" + 0.015*"fuel" + 0.011*"wind" + 0.011*"gas" + 0.010*"coal" + 0.008*"big" + 0.008*"efficiency" + 0.007*"clean" + 0.007*"natural" + 0.007*"waste" + 0.007*"electric" + 0.007*"cost" + 0.006*"battery

In [25]:
number_of_topics=60 # adjust this to alter the number of topics
words=20 #adjust this to alter the number of words output for the topic below

In [26]:
%%time
# runs LDA using Mallet from gensim using the number_of_topics specified above
ldamallet60 = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=number_of_topics, id2word=dictionary)

Wall time: 2min 1s


In [27]:
ldamallet60 = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=number_of_topics, id2word=dictionary)

In [28]:
# output the topics
ldamallet60.show_topics(num_topics=number_of_topics,num_words=words)

[(0,
  '0.061*"earth" + 0.034*"planet" + 0.027*"space" + 0.017*"mars" + 0.014*"moon" + 0.013*"surface" + 0.012*"system" + 0.011*"planets" + 0.009*"solar" + 0.009*"atmosphere" + 0.009*"ice" + 0.007*"water" + 0.007*"nasa" + 0.006*"clouds" + 0.006*"satellite" + 0.006*"sun" + 0.005*"ago" + 0.005*"picture" + 0.005*"satellites" + 0.005*"miles"'),
 (1,
  '0.092*"music" + 0.039*"sound" + 0.020*"hear" + 0.020*"play" + 0.018*"song" + 0.017*"sounds" + 0.011*"singing" + 0.011*"dance" + 0.011*"piece" + 0.010*"noise" + 0.009*"listening" + 0.009*"listen" + 0.009*"la" + 0.009*"playing" + 0.008*"sing" + 0.008*"musical" + 0.007*"piano" + 0.007*"songs" + 0.007*"hearing" + 0.007*"audience"'),
 (2,
  '0.054*"system" + 0.025*"model" + 0.022*"systems" + 0.015*"project" + 0.015*"open" + 0.014*"technology" + 0.013*"innovation" + 0.012*"working" + 0.012*"build" + 0.011*"start" + 0.011*"create" + 0.010*"problem" + 0.010*"big" + 0.010*"models" + 0.008*"process" + 0.008*"source" + 0.008*"technologies" + 0.008*"bui

In [29]:
number_of_topics=50 # adjust this to alter the number of topics
words=20 #adjust this to alter the number of words output for the topic below

In [30]:
%%time
# runs LDA using Mallet from gensim using the number_of_topics specified above
ldamallet50 = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=number_of_topics, id2word=dictionary)

Wall time: 2min 3s


In [31]:
ldamallet50 = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=number_of_topics, id2word=dictionary)

In [32]:
# output the topics
ldamallet50.show_topics(num_topics=number_of_topics,num_words=words)

[(0,
  '0.045*"cancer" + 0.035*"cells" + 0.030*"body" + 0.026*"blood" + 0.020*"heart" + 0.012*"skin" + 0.011*"cell" + 0.010*"surgery" + 0.010*"disease" + 0.010*"tissue" + 0.009*"stem" + 0.008*"patient" + 0.007*"organs" + 0.007*"tumor" + 0.007*"breast" + 0.007*"muscle" + 0.006*"bone" + 0.006*"system" + 0.005*"organ" + 0.005*"immune"'),
 (1,
  '0.029*"animals" + 0.023*"species" + 0.018*"animal" + 0.017*"human" + 0.015*"humans" + 0.012*"dog" + 0.010*"birds" + 0.008*"evolved" + 0.008*"dogs" + 0.008*"male" + 0.007*"evolution" + 0.007*"creatures" + 0.006*"behavior" + 0.006*"ants" + 0.006*"eggs" + 0.006*"males" + 0.005*"bird" + 0.005*"evolutionary" + 0.005*"natural" + 0.005*"insects"'),
 (2,
  '0.081*"children" + 0.045*"child" + 0.030*"family" + 0.022*"parents" + 0.020*"kids" + 0.019*"baby" + 0.017*"mother" + 0.016*"young" + 0.016*"age" + 0.014*"families" + 0.013*"born" + 0.013*"care" + 0.013*"babies" + 0.010*"older" + 0.010*"lives" + 0.010*"birth" + 0.009*"home" + 0.008*"adults" + 0.007*"mot

In [33]:
number_of_topics=30 # adjust this to alter the number of topics
words=20 #adjust this to alter the number of words output for the topic below

In [34]:
%%time
# runs LDA using Mallet from gensim using the number_of_topics specified above
ldamallet30 = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=number_of_topics, id2word=dictionary)

Wall time: 1min 55s


In [35]:
ldamallet30 = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=number_of_topics, id2word=dictionary)

In [36]:
# output the topics
ldamallet30.show_topics(num_topics=number_of_topics,num_words=words)

[(0,
  '0.033*"health" + 0.021*"care" + 0.014*"disease" + 0.013*"medical" + 0.013*"patients" + 0.010*"drug" + 0.010*"drugs" + 0.009*"hospital" + 0.009*"treatment" + 0.009*"doctors" + 0.008*"doctor" + 0.008*"percent" + 0.007*"hiv" + 0.007*"patient" + 0.007*"medicine" + 0.005*"high" + 0.005*"diseases" + 0.005*"early" + 0.004*"symptoms" + 0.004*"risk"'),
 (1,
  '0.046*"human" + 0.020*"humans" + 0.017*"animals" + 0.015*"species" + 0.010*"animal" + 0.009*"ago" + 0.008*"nature" + 0.008*"evolution" + 0.008*"dog" + 0.007*"male" + 0.006*"natural" + 0.006*"beings" + 0.006*"million" + 0.006*"intelligence" + 0.006*"birds" + 0.005*"000" + 0.005*"question" + 0.005*"babies" + 0.005*"dogs" + 0.005*"female"'),
 (2,
  '0.018*"war" + 0.013*"power" + 0.013*"political" + 0.012*"country" + 0.007*"government" + 0.007*"democracy" + 0.007*"states" + 0.006*"politics" + 0.006*"military" + 0.006*"president" + 0.006*"united" + 0.005*"american" + 0.005*"conflict" + 0.005*"peace" + 0.005*"civil" + 0.005*"change" + 0

In [37]:
number_of_topics=20 # adjust this to alter the number of topics
words=20 #adjust this to alter the number of words output for the topic below

In [38]:
%%time
# runs LDA using Mallet from gensim using the number_of_topics specified above
ldamallet20 = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=number_of_topics, id2word=dictionary)

Wall time: 2min 13s


In [39]:
ldamallet20 = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=number_of_topics, id2word=dictionary)

In [40]:
# output the topics
ldamallet20.show_topics(num_topics=number_of_topics,num_words=words)

[(0,
  '0.010*"problem" + 0.009*"study" + 0.009*"question" + 0.007*"research" + 0.007*"percent" + 0.007*"change" + 0.007*"important" + 0.006*"answer" + 0.006*"science" + 0.005*"data" + 0.005*"hard" + 0.005*"group" + 0.005*"decisions" + 0.005*"problems" + 0.005*"questions" + 0.004*"based" + 0.004*"social" + 0.004*"choice" + 0.004*"evidence" + 0.004*"results"'),
 (1,
  '0.045*"women" + 0.024*"men" + 0.017*"woman" + 0.014*"black" + 0.010*"girls" + 0.010*"sex" + 0.009*"white" + 0.009*"talk" + 0.009*"young" + 0.007*"girl" + 0.007*"man" + 0.006*"female" + 0.006*"gender" + 0.006*"male" + 0.006*"feel" + 0.005*"change" + 0.005*"sexual" + 0.004*"person" + 0.004*"told" + 0.004*"love"'),
 (2,
  '0.013*"game" + 0.009*"video" + 0.008*"move" + 0.007*"hand" + 0.007*"body" + 0.006*"games" + 0.006*"play" + 0.006*"show" + 0.005*"side" + 0.005*"minutes" + 0.005*"fly" + 0.005*"left" + 0.005*"front" + 0.005*"air" + 0.005*"feet" + 0.004*"feel" + 0.004*"playing" + 0.004*"moment" + 0.004*"hands" + 0.004*"walk"

## Convert to Gensim model format
Convert the Mallet model to gensim format.

In [41]:
gensimmodel100 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet100)

In [42]:
print(gensimmodel100)

LdaModel(num_terms=23039, num_topics=100, decay=0.5, chunksize=2000)


In [43]:
gensimmodel60 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet60)

In [44]:
print(gensimmodel60)

LdaModel(num_terms=23039, num_topics=60, decay=0.5, chunksize=2000)


In [45]:
gensimmodel50 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet50)

In [46]:
print(gensimmodel50)

LdaModel(num_terms=23039, num_topics=50, decay=0.5, chunksize=2000)


In [47]:
gensimmodel30 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet30)

In [48]:
print(gensimmodel30)

LdaModel(num_terms=23039, num_topics=30, decay=0.5, chunksize=2000)


In [49]:
gensimmodel20 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet20)

In [50]:
print(gensimmodel20)

LdaModel(num_terms=23039, num_topics=20, decay=0.5, chunksize=2000)


## Get a coherence score

In [51]:
coherencemodel = CoherenceModel(model=gensimmodel100, texts=doc_clean, dictionary=dictionary, coherence='c_v')
print (coherencemodel.get_coherence())

0.5455256120705014


In [52]:
coherencemodel = CoherenceModel(model=gensimmodel60, texts=doc_clean, dictionary=dictionary, coherence='c_v')
print (coherencemodel.get_coherence())

0.566109653634188


In [53]:
coherencemodel = CoherenceModel(model=gensimmodel50, texts=doc_clean, dictionary=dictionary, coherence='c_v')
print (coherencemodel.get_coherence())

0.5608885323446677


In [54]:
coherencemodel = CoherenceModel(model=gensimmodel30, texts=doc_clean, dictionary=dictionary, coherence='c_v')
print (coherencemodel.get_coherence())

0.5403421787381771


In [55]:
coherencemodel = CoherenceModel(model=gensimmodel20, texts=doc_clean, dictionary=dictionary, coherence='c_v')
print (coherencemodel.get_coherence())

0.5204559197192618


## Preview a document

Preview a document - you can change the doc_id to view another document.

In [80]:
doc_id = 2476 # index of document to explore
print(re.sub('\s+', ' ', document_list[doc_id])) 

I want to talk to you today about prosperity, about our hopes for a shared and lasting prosperity. And not just us, but the two billion people worldwide who are still chronically undernourished. And hope actually is at the heart of this. In fact, the Latin word for hope is at the heart of the word prosperity. "Pro-speras," "speras," hope — in accordance with our hopes and expectations. The irony is, though, that we have cashed-out prosperity almost literally in terms of money and economic growth. And we've grown our economies so much that we now stand in a real danger of undermining hope — running down resources, cutting down rainforests, spilling oil into the Gulf of Mexico, changing the climate — and the only thing that has actually remotely slowed down the relentless rise of carbon emissions over the last two to three decades is recession. And recession, of course, isn't exactly a recipe for hope either, as we're busy finding out. So we're caught in a kind of trap. It's a dilemma, a

## Output the distribution of topics for the document

The next cell outputs the distribution of topics on the document specified above.

In [81]:
len(doc_term_matrix[doc_id])

709

## Find similar documents
This will find the 5 most similar documents to the document specified above based on their topic distribution.

In [82]:
# gensimmodel30[doc_term_matrix] below represents the documents in the corpus in LDA vector space
lda_index = similarities.MatrixSimilarity(gensimmodel30[doc_term_matrix])

# query for our doc_id from above
similarity_index = lda_index[gensimmodel30[doc_term_matrix[doc_id]]]

# Sort the similarity index
similarity_index = sorted(enumerate(similarity_index), key=lambda item: -item[1])

for i in range(1,4): 
    document_id, similarity_score = similarity_index[i]

    print('Document Index:',document_id)
    print('Document:', document_list2[document_id])
    print('Similarity Score:',similarity_score)
    
    print(re.sub('\s+', ' ', document_list[document_id][:500]), '...') # preview first 500 characters

    document_topics = gensimmodel30[doc_term_matrix[document_id]]
    document_topics = sorted(document_topics, key=lambda x: x[1], reverse=True)
    for topic, prop in document_topics:
        topic_words = [word[0] for word in gensimmodel30.show_topic(topic, 10)]
        print ("%.2f" % prop, topic, topic_words)
    
    print()

Document Index: 1738
Document: P:\Downloads\ted-transcripts\transcripts\2010-07-16-nic_marks_the_happy_planet_index.txt
Similarity Score: 0.940459
Martin Luther King did not say, "I have a nightmare," when he inspired the civil rights movements. He said, "I have a dream." And I have a dream. I have a dream that we can stop thinking that the future will be a nightmare, and this is going to be a challenge, because, if you think of every major blockbusting film of recent times, nearly all of its visions for humanity are apocalyptic. I think this film is one of the hardest watches of modern times, "The Road." It's a beautiful piece of filmmaki ...
0.29 23 ['countries', 'china', 'economic', 'global', 'country', 'social', 'states', 'growth', 'economy', 'united']
0.09 20 ['change', 'future', 'percent', 'problem', 'problems', 'end', 'age', 'level', 'past', 'changing']
0.08 27 ['energy', 'water', 'climate', 'carbon', 'oil', 'air', 'power', 'percent', 'waste', 'solar']
0.07 25 ['feel', 'love', '

In [83]:
doc_id2 = 2037 # index of document to explore
print(re.sub('\s+', ' ', document_list[doc_id2])) 

So when people voice fears of artificial intelligence, very often, they invoke images of humanoid robots run amok. You know? Terminator? You know, that might be something to consider, but that's a distant threat. Or, we fret about digital surveillance with metaphors from the past. "1984," George Orwell's "1984," it's hitting the bestseller lists again. It's a great book, but it's not the correct dystopia for the 21st century. What we need to fear most is not what artificial intelligence will do to us on its own, but how the people in power will use artificial intelligence to control us and to manipulate us in novel, sometimes hidden, subtle and unexpected ways. Much of the technology that threatens our freedom and our dignity in the near-term future is being developed by companies in the business of capturing and selling our data and our attention to advertisers and others: Facebook, Google, Amazon, Alibaba, Tencent. Now, artificial intelligence has started bolstering their business as

In [84]:
# gensimmodel30[doc_term_matrix] below represents the documents in the corpus in LDA vector space
lda_index = similarities.MatrixSimilarity(gensimmodel30[doc_term_matrix])

# query for our doc_id from above
similarity_index = lda_index[gensimmodel30[doc_term_matrix[doc_id2]]]

# Sort the similarity index
similarity_index = sorted(enumerate(similarity_index), key=lambda item: -item[1])

for i in range(1,4): 
    document_id, similarity_score = similarity_index[i]

    print('Document Index:',document_id)
    print('Document:', document_list2[document_id])
    print('Similarity Score:',similarity_score)
    
    print(re.sub('\s+', ' ', document_list[document_id][:500]), '...') # preview first 500 characters

    document_topics2 = gensimmodel30[doc_term_matrix[document_id]]
    document_topics2 = sorted(document_topics2, key=lambda x: x[1], reverse=True)
    for topic, prop in document_topics2:
        topic_words = [word[0] for word in gensimmodel30.show_topic(topic, 10)]
        print ("%.2f" % prop, topic, topic_words)
    
    print()

Document Index: 1032
Document: P:\Downloads\ted-transcripts\transcripts\2018-09-26-tamas_kocsis_the_case_for_a_decentralized_internet.txt
Similarity Score: 0.96096766
Three years ago, I started building a decentralized web because I was worried about the future of our internet. The current internet we are using is about gatekeepers. If you want to reach something on the web, then you need to go through multiple middlemen. First, a domain name server, then a server hosting company, which usually points you to a third party, to a web hosting service. And this happens every time you want to reach a website on the web. But these gatekeepers are vulnerable to in ...
0.41 5 ['data', 'information', 'computer', 'internet', 'technology', 'online', 'phone', 'media', 'digital', 'google']
0.09 24 ['law', 'police', 'violence', 'justice', 'prison', 'case', 'rights', 'public', 'crime', 'system']
0.08 2 ['war', 'power', 'political', 'country', 'government', 'democracy', 'states', 'politics', 'military

In [85]:
len(doc_term_matrix[doc_id2])

658

In [78]:
doc_term_matrix[doc_id2]

[(0, 3),
 (3, 2),
 (18, 1),
 (20, 1),
 (24, 5),
 (26, 1),
 (32, 2),
 (33, 3),
 (35, 2),
 (38, 1),
 (43, 1),
 (64, 2),
 (80, 2),
 (85, 1),
 (93, 1),
 (96, 3),
 (100, 1),
 (102, 2),
 (111, 2),
 (112, 3),
 (113, 2),
 (114, 3),
 (115, 18),
 (117, 1),
 (128, 2),
 (132, 1),
 (134, 2),
 (163, 1),
 (170, 1),
 (171, 2),
 (173, 1),
 (174, 2),
 (189, 1),
 (192, 1),
 (205, 7),
 (212, 1),
 (217, 1),
 (223, 1),
 (226, 1),
 (234, 2),
 (235, 2),
 (238, 1),
 (241, 4),
 (243, 1),
 (259, 1),
 (281, 2),
 (293, 1),
 (298, 1),
 (306, 1),
 (311, 1),
 (312, 1),
 (313, 3),
 (333, 5),
 (348, 1),
 (349, 6),
 (354, 1),
 (356, 6),
 (360, 1),
 (362, 3),
 (367, 3),
 (382, 1),
 (386, 2),
 (402, 1),
 (406, 1),
 (408, 2),
 (410, 1),
 (415, 1),
 (428, 3),
 (431, 1),
 (432, 1),
 (435, 1),
 (439, 1),
 (440, 1),
 (441, 1),
 (457, 1),
 (466, 2),
 (487, 1),
 (490, 1),
 (491, 1),
 (502, 1),
 (505, 2),
 (507, 1),
 (508, 1),
 (513, 8),
 (556, 1),
 (563, 1),
 (568, 1),
 (573, 1),
 (583, 1),
 (587, 1),
 (613, 6),
 (617, 1),
 (635